In [1]:
!apt install unzip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  zip
The following NEW packages will be installed:
  unzip
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 166 kB of archives.
After this operation, 388 kB of additional disk space will be used.
Get:1 http://deb.debian.org/debian bookworm/main amd64 unzip amd64 6.0-28 [166 kB]
Fetched 166 kB in 10s (16.6 kB/s)     
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package unzip.
(Reading database ... 23901 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-28_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8Unpacking unzip (6.0-28) ...
Setting up unzip (6.0-28) ...[ 40%] [#######################...................................]

In [2]:
!curl -L -o complete-include-pickle-all-aug.zip https://www.kaggle.com/api/v1/datasets/download/riceyy/complete-include-pickle-all-aug
!curl -L -o complete-include-pickle-hflip.zip https://www.kaggle.com/api/v1/datasets/download/riceyy/complete-include-pickle-hflip
!curl -L -o complete-include-pickle-no-aug-480.zip https://www.kaggle.com/api/v1/datasets/download/riceyy/complete-include-pickle-no-aug-480
!curl -L -o complete-include-pickle-random-crop.zip https://www.kaggle.com/api/v1/datasets/download/riceyy/complete-include-pickle-random-crop
!curl -L -o complete-include-pickle-random-rotate.zip https://www.kaggle.com/api/v1/datasets/download/riceyy/complete-include-pickle-random-rotate

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:05 --:--:--     0
100 1691M  100 1691M    0     0  53.5M      0  0:00:31  0:00:31 --:--:--  103M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:05 --:--:--     0
100 1914M  100 1914M    0     0  61.5M      0  0:00:31  0:00:31 --:--:--  116M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:05 --:--:--     0
100 1908M  100 1908M    0     0  64.1M      0  0:00:29  0:00:29 --:--:--  118M
  % Total    % Received % Xferd  Average Speed   Tim

In [3]:
!unzip complete-include-pickle-all-aug.zip -d /home/complete-include-pickle-all-aug
!unzip complete-include-pickle-hflip.zip -d /home/complete-include-pickle-hflip
!unzip complete-include-pickle-no-aug-480.zip -d /home/complete-include-pickle-no-aug-480
!unzip complete-include-pickle-random-crop.zip -d /home/complete-include-pickle-random-crop
!unzip complete-include-pickle-random-rotate.zip -d /home/complete-include-pickle-random-rotate

Archive:  complete-include-pickle-all-aug.zip
  inflating: /home/complete-include-pickle-all-aug/Test_data/tensor_0.pt  
  inflating: /home/complete-include-pickle-all-aug/Test_data/tensor_1.pt  
  inflating: /home/complete-include-pickle-all-aug/Test_data/tensor_10.pt  
  inflating: /home/complete-include-pickle-all-aug/Test_data/tensor_11.pt  
  inflating: /home/complete-include-pickle-all-aug/Test_data/tensor_12.pt  
  inflating: /home/complete-include-pickle-all-aug/Test_data/tensor_13.pt  
  inflating: /home/complete-include-pickle-all-aug/Test_data/tensor_14.pt  
  inflating: /home/complete-include-pickle-all-aug/Test_data/tensor_15.pt  
  inflating: /home/complete-include-pickle-all-aug/Test_data/tensor_16.pt  
  inflating: /home/complete-include-pickle-all-aug/Test_data/tensor_17.pt  
  inflating: /home/complete-include-pickle-all-aug/Test_data/tensor_18.pt  
  inflating: /home/complete-include-pickle-all-aug/Test_data/tensor_19.pt  
  inflating: /home/complete-include-pickle-a

In [5]:
import torch
import torchvision.transforms as transforms
import torch.nn as nn
from torchvision import models
from torchvision.transforms.functional import to_tensor
from torch.amp import autocast, GradScaler

import os
import numpy as np
import subprocess
from PIL import Image
import shutil
import torch.nn.functional as F

# %matplotlib inline
# from matplotlib import pyplot as plt
# import matplotlib.gridspec as gridspec
# os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [6]:
device = 'cuda'

In [7]:
def read_pickle(path) -> torch.tensor:
    return torch.load(path)

def get_all_file_paths(path) -> list:
    path_list = []
    for top, dirs, files in os.walk(path):
        for nm in files:
            path_list.append(os.path.join(top, nm))
    return path_list

def preload_tensors(path_list, device):
    data = []
    for path in path_list:
        tensor = torch.load(path)  # Load tensor from disk
        video, label = tensor['video'].to(device), tensor['label'].to(device)
        data.append((video.to(torch.float16), label))
    return data

In [8]:
import torch

def save_checkpoint(state, filename="checkpoint.pth"):
    """Save model and optimizer state to a file."""
    torch.save(state, filename)

def load_checkpoint(filename="checkpoint.pth"):
    """Load model and optimizer state from a file."""
    checkpoint = torch.load(filename)
    return checkpoint

In [9]:
class Trainer:
    def __init__(self, model, device, optimizer, loss_fn, scheduler, save=False):
        self.device = device
        self.scaler = GradScaler()  # Automatic Mixed Precision
        self.optimizer = optimizer
        self.scheduler = scheduler  # Add scheduler
        self.loss_fn = loss_fn
        self.model = model
        self.save = save

    def train_step_pickle(self, epoch, preloaded_data):
        self.model.train()
        train_loss, total_correct, total_samples = 0, 0, 0

        for X, y in preloaded_data:
            # Forward pass with AMP
            with autocast('cuda'):
                y_pred = self.model(X)
                loss = self.loss_fn(y_pred, y)
                train_loss += loss.item()

            # Backward pass with scaling
            self.scaler.scale(loss).backward()
            self.scaler.step(self.optimizer)
            self.scaler.update()
            self.optimizer.zero_grad()

            # Accuracy computation
            y_pred_class = torch.argmax(y_pred, dim=1)
            total_correct += (y_pred_class == y).sum().item()
            total_samples += y.size(0)

        acc = total_correct * 100 / total_samples
        train_loss /= len(preloaded_data)
        print(f"Epoch: {epoch} | Train Loss: {train_loss:.4f} | Accuracy: {acc:.2f}")

        # Step scheduler if it's not ReduceLROnPlateau
        if not isinstance(self.scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
            self.scheduler.step()

    def test_step_pickle(self, epoch,preloaded_data):
        self.model.eval()
        test_loss, acc = 0, 0
        total_correct = 0
        total_samples = 0

        with torch.no_grad():
            for X, y in preloaded_data:
                # Use AMP for inference
                with autocast('cuda'):
                    y_pred = self.model(X)
                    loss = self.loss_fn(y_pred, y)
                    test_loss += loss.item()

                # Compute predictions and accuracy
                y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
                total_correct += (y_pred_class == y).sum().item()
                total_samples += y.size(0)

            acc = total_correct * 100 / total_samples
            test_loss = test_loss / len(preloaded_data)
            print(f"Epoch: {epoch} | Test Loss: {test_loss:.4f} | Accuracy: {acc:.2f}")
            
            # Step scheduler if ReduceLROnPlateau
            if isinstance(self.scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                self.scheduler.step(test_loss)  # Use validation loss as the metric
            
            if self.save:
                # Define a threshold for saving the model
                save_threshold = 75.0
            
                if acc > save_threshold:
                    print(f"Accuracy {acc:.2f}% exceeded the threshold of {save_threshold}%. Attempting to save the model...")
            
                    # Check available storage space before saving
                    try:
                        import shutil  # To check disk space
                        total, used, free = shutil.disk_usage("/")
                        free_gb = free / (1024 ** 3)  # Convert to GB
            
                        # Set a minimum free space threshold (e.g., 1 GB)
                        if free_gb < 1.0:
                            print("Insufficient storage space available. Skipping model save.")
                        else:
                            # Save the model
                            save_checkpoint({
                                'epoch': epoch,
                                'model_state_dict': self.model.state_dict(),
                                'optimizer_state_dict': self.optimizer.state_dict(),
                                'best_metric': acc
                            }, filename=f"{model.name}_model={acc:.2f}_epoch={epoch}_real.pth")
                            print(f"Model saved successfully at epoch {epoch}.")
                    except Exception as e:
                        print(f"Error while checking storage or saving the model: {e}")
                else:
                    print(f"Accuracy {acc:.2f}% did not exceed the threshold of {save_threshold}%. Model not saved.")


In [10]:
train_preloaded = []
train_path = ['/home/complete-include-pickle-all-aug/Train_data','/home/complete-include-pickle-hflip/Train_data',
              '/home/complete-include-pickle-no-aug-480/Train_data','/home/complete-include-pickle-random-crop/Train_data',
              '/home/complete-include-pickle-random-rotate/Train_data'
             ]

# for path in train_path:
#     train_preloaded.extend(preload_tensors(get_all_file_paths(path), device='cuda:0'))


test_preloaded_all = []
test_path_all = ['/home/complete-include-pickle-all-aug/Test_data','/home/complete-include-pickle-hflip/Test_data',
                '/home/complete-include-pickle-no-aug-480/Test_data','/home/complete-include-pickle-random-crop/Test_data',
                '/home/complete-include-pickle-random-rotate/Test_data'
                ]

# for path in test_path_all:
#     test_preloaded_all.extend(preload_tensors(get_all_file_paths(path), device='cuda:0'))

# test_preloaded_unaug = preload_tensors(get_all_file_paths('/home/include-interpolated-300p/ISL_pickle/Test_data'), device='cuda:0')



# train_preloaded = preload_tensors(get_all_file_paths('/kaggle/input/include-interpolated-cv-300p-random-crop/Train_data'), device='cuda:0')
# test_preloaded = preload_tensors(get_all_file_paths('/kaggle/input/include-interpolated-cv-300p-random-crop/Test_data'), device='cuda:0')

In [11]:
train_path_lists = []
for path in train_path:
    train_path_lists.extend(get_all_file_paths(path))

In [12]:
test_path_lists = []
for path in test_path_all:
    test_path_lists.extend(get_all_file_paths(path))

In [13]:
len(train_path_lists)

1045

In [14]:
len(test_path_lists)

295

In [15]:
def path_shuffler(path_list,seed=42):
    np.random.seed(seed)
    np.random.shuffle(path_list)

In [16]:
path_shuffler(train_path_lists)
path_shuffler(test_path_lists)

In [17]:
import torch
from torch.amp import autocast, GradScaler

class Trainer:
    def __init__(self,model,device,optimizer,loss_fn, scheduler,train_path_list,test_path_list,save=False):
        self.device = device
        self.scaler = GradScaler(device)  # Initialize GradScaler
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.scheduler = scheduler
        self.train_path_list = train_path_list
        self.test_path_list = test_path_list
        self.model = model
        self.save = save

    def train_step_pickle(self,epoch):
        self.model.train()
        train_loss, acc = 0, 0
        total_correct = 0
        total_samples = 0
        

        for path in self.train_path_list:
            tensor = read_pickle(path=path)
            X = tensor['video'].to(torch.float16)
            y = tensor['label'].to(self.device)

             # Slice X into 4 parts
            if X.size(0) == 16:
                X_parts = torch.chunk(X, 4, dim=0)
                y_parts = torch.chunk(y, 4, dim=0)
            else:
                X_parts = [X]
                y_parts = [y]

            for X_part, y_part in zip(X_parts, y_parts):
                # Automatic Mixed Precision (AMP)
                with autocast(device_type=device):
                    y_pred = self.model(X_part.to(self.device))
                    loss = self.loss_fn(y_pred, y_part)
                    train_loss += loss.item()

                # Scale the loss and call backward
                self.scaler.scale(loss).backward()
                # Update parameters with scaled gradients
                self.scaler.step(self.optimizer)
                # Update scaler for next iteration
                self.scaler.update()

                self.optimizer.zero_grad()

                # Compute predictions and accuracy
                y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
                total_correct += (y_pred_class == y_part).sum().item()
                total_samples += y_part.size(0)

                del X_part
                del y_part

        acc = total_correct * 100 / total_samples
        train_loss = train_loss / len(self.train_path_list)
        print(f"Epoch: {epoch} | Train Loss: {train_loss:.4f} | Accuracy: {acc:.2f}")

            # Step scheduler if it's not ReduceLROnPlateau
        if not isinstance(self.scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
            self.scheduler.step()


    def test_step_pickle(self, epoch):
        self.model.eval()
        test_loss, acc = 0, 0
        total_correct = 0
        total_samples = 0

        with torch.no_grad():
            for path in self.test_path_list:
                tensor = read_pickle(path=path)
                X = tensor['video'].to(torch.float16)
                y = tensor['label'].to(self.device)

                # Slice X into 4 parts
                if X.size(0) == 16:
                    X_parts = torch.chunk(X, 4, dim=0)
                    y_parts = torch.chunk(y, 4, dim=0)
                else:
                    X_parts = [X]
                    y_parts = [y]

                for X_part, y_part in zip(X_parts, y_parts):
                    # Use AMP for inference
                    with autocast('cuda'):
                        y_pred = self.model(X_part.to(self.device))
                        loss = self.loss_fn(y_pred, y_part)
                        test_loss += loss.item()

                    # Compute predictions and accuracy
                    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
                    total_correct += (y_pred_class == y_part).sum().item()
                    total_samples += y_part.size(0)

                    del X_part
                    del y_part

            acc = total_correct * 100 / total_samples
            test_loss = test_loss / len(self.test_path_list)
            print(f"Epoch: {epoch} | Test Loss: {test_loss:.4f} | Accuracy: {acc:.2f}")
            print("************************")

            # Step scheduler if ReduceLROnPlateau
            if isinstance(self.scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                self.scheduler.step(test_loss)  # Use validation loss as the metric

            if self.save:
                # Define a threshold for saving the model
                save_threshold = 75.0
            
                if acc > save_threshold:
                    print(f"Accuracy {acc:.2f}% exceeded the threshold of {save_threshold}%. Attempting to save the model...")
            
                    # Check available storage space before saving
                    try:
                        import shutil  # To check disk space
                        total, used, free = shutil.disk_usage("/")
                        free_gb = free / (1024 ** 3)  # Convert to GB
            
                        # Set a minimum free space threshold (e.g., 1 GB)
                        if free_gb < 1.0:
                            print("Insufficient storage space available. Skipping model save.")
                        else:
                            # Save the model
                            save_checkpoint({
                                'epoch': epoch,
                                'model_state_dict': self.model.state_dict(),
                                'optimizer_state_dict': self.optimizer.state_dict(),
                                'best_metric': acc
                            }, filename=f"{self.model.name}_model={acc:.2f}_epoch={epoch}_real.pth")
                            print(f"Model saved successfully at epoch {epoch}.")
                    except Exception as e:
                        print(f"Error while checking storage or saving the model: {e}")
                else:
                    print(f"Accuracy {acc:.2f}% did not exceed the threshold of {save_threshold}%. Model not saved.")

In [18]:
# len(train_preloaded), len(test_preloaded_all), len(test_preloaded_unaug)

In [19]:
import torch
import torch.nn as nn
from torchvision import models

# CNN Model
class r3d(nn.Module):
    def __init__(self, num_classes, name='r3d'):
        super(r3d, self).__init__()

        # model_name
        self.name= name

        # CNN (ResNet - 3D)
        self.cnn = models.video.r3d_18(pretrained=True)  # Pretrained 3D ResNet
        self.cnn.fc = nn.Identity()  # Remove final classification layer

        # Fully connected layer for classification
        self.fc = nn.Linear(512, num_classes)  # r3d_18 final layer outputs 512 features

    def forward(self, x):
        """
        Forward pass for the model.

        Args:
        - x: Input tensor with shape [Batch, Sequence, Channels, Height, Width]

        Returns:
        - output: Model predictions with shape [Batch, Num_Classes]
        """
        batch_size, seq_len, c, h, w = x.size()  # x: [Batch, Sequence, Channels, Height, Width]

        # Reshape sequence as depth for 3D CNN
        x = x.permute(0, 2, 1, 3, 4)  # [Batch, Channels, Depth (Seq_Len), Height, Width]

        # Extract features using CNN
        features = self.cnn(x)  # [Batch, 512, 1, 1, 1]
        features = features.view(features.size(0), -1)  # Flatten: [Batch, 512]

        # Pass through the fully connected layer
        output = self.fc(features)  # [Batch, Num_Classes]
        return output

In [20]:
import torch
import torch.nn as nn
from torchvision import models

# CNN + GRU Model with Dropouts
class r18gru(nn.Module):
    def __init__(self, num_classes, dropout, name='r18gru'):
        super(r18gru, self).__init__()

        # model_name
        self.name= name

        # Pre-trained CNN (e.g., ResNet)
        self.cnn = models.resnet18(pretrained=True)
        self.cnn = nn.Sequential(*list(self.cnn.children())[:-1])  # Remove the final classification layer

        # Dropout after CNN features
        self.cnn_dropout = nn.Dropout(p=dropout)

        # GRU for temporal modeling
        self.gru = nn.GRU(input_size=512, hidden_size=256, num_layers=2, batch_first=True, dropout=dropout)

        # Fully connected layer for classification
        self.fc = nn.Linear(256, num_classes)

        # Dropout before final FC layer
        self.fc_dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        """
        Forward pass for the model.

        Args:
        - x: Input tensor with shape [Batch, Sequence, Channels, Height, Width]

        Returns:
        - output: Model predictions with shape [Batch, Num_Classes]
        """
        batch_size, seq_len, c, h, w = x.size()  # x: [Batch, Sequence, Channels, Height, Width]

        # Reshape input for CNN processing
        x = x.view(batch_size * seq_len, c, h, w)  # Combine Batch and Sequence dimensions: [Batch * Sequence, Channels, Height, Width]

        # Extract features using CNN
        features = self.cnn(x)  # [Batch * Sequence, 512, 1, 1] (ResNet-18 output)
        features = features.view(features.size(0), -1)  # Flatten: [Batch * Sequence, 512]

        # Apply CNN dropout
        features = self.cnn_dropout(features)

        # Reshape features back to sequence format for GRU
        features = features.view(batch_size, seq_len, -1)  # [Batch, Sequence, Features]

        # Pass the sequence of features through the GRU
        gru_out, _ = self.gru(features)  # [Batch, Sequence, Hidden_Size]

        # Take the output of the last time step
        final_output = gru_out[:, -1, :]  # [Batch, Hidden_Size]

        # Apply dropout before the fully connected layer
        final_output = self.fc_dropout(final_output)

        # Pass through the fully connected layer
        output = self.fc(final_output)  # [Batch, Num_Classes]
        return output

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# CNN + LSTM Model
class r18lstm(nn.Module):
    def __init__(self, num_classes, name='r18lstm'):
        super(r18lstm, self).__init__()

        # model_name
        self.name= name

        # Pre-trained CNN (e.g., ResNet)
        self.cnn = models.resnet18(pretrained=True)
        self.cnn = nn.Sequential(*list(self.cnn.children())[:-1])  # Remove the final classification layer

        # LSTM for temporal modeling
        self.lstm = nn.LSTM(input_size=512, hidden_size=256, num_layers=2, batch_first=True)

        # Fully connected layer for classification
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        """
        Forward pass for the model.

        Args:
        - x: Input tensor with shape [Batch, Sequence, Channels, Height, Width]

        Returns:
        - output: Model predictions with shape [Batch, Num_Classes]
        """
        batch_size, seq_len, c, h, w = x.size()  # x: [Batch, Sequence, Channels, Height, Width]

        # Reshape input for CNN processing
        x = x.view(batch_size * seq_len, c, h, w)  # Combine Batch and Sequence dimensions: [Batch * Sequence, Channels, Height, Width]

        # Extract features using CNN
        features = self.cnn(x)  # [Batch * Sequence, 512, 1, 1] (ResNet-18 output)
        features = features.view(features.size(0), -1)  # Flatten: [Batch * Sequence, 512]

        # Reshape features back to sequence format for LSTM
        features = features.view(batch_size, seq_len, -1)  # [Batch, Sequence, Features]

        # Pass the sequence of features through the LSTM
        lstm_out, _ = self.lstm(features)  # [Batch, Sequence, Hidden_Size]

        # Take the output of the last time step
        final_output = lstm_out[:, -1, :]  # [Batch, Hidden_Size]

        # Pass through the fully connected layer
        output = self.fc(final_output)  # [Batch, Num_Classes]
        return output

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# CNN + GRU Model with Dropouts
class mv3gru(nn.Module):
    def __init__(self, num_classes, dropout, name='mv3gru'):
        super(mv3gru, self).__init__()

        # model_name
        self.name= name

        # Pre-trained CNN (e.g., ResNet)
        self.cnn = models.mobilenet_v3_large(pretrained=True)
        self.cnn = nn.Sequential(*list(self.cnn.children())[:-1])  # Remove the final classification layer

        # Dropout after CNN features
        self.cnn_dropout = nn.Dropout(p=dropout)

        # GRU for temporal modeling
        self.gru = nn.GRU(input_size=960, hidden_size=256, num_layers=2, batch_first=True, dropout=dropout)

        # Fully connected layer for classification
        self.fc = nn.Linear(256, num_classes)

        # Dropout before final FC layer
        self.fc_dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        """
        Forward pass for the model.

        Args:
        - x: Input tensor with shape [Batch, Sequence, Channels, Height, Width]

        Returns:
        - output: Model predictions with shape [Batch, Num_Classes]
        """
        batch_size, seq_len, c, h, w = x.size()  # x: [Batch, Sequence, Channels, Height, Width]

        # Reshape input for CNN processing
        x = x.view(batch_size * seq_len, c, h, w)  # Combine Batch and Sequence dimensions: [Batch * Sequence, Channels, Height, Width]

        # Extract features using CNN
        features = self.cnn(x)  # [Batch * Sequence, 512, 1, 1] (ResNet-18 output)
        features = features.view(features.size(0), -1)  # Flatten: [Batch * Sequence, 512]

        # Apply CNN dropout
        features = self.cnn_dropout(features)

        # Reshape features back to sequence format for GRU
        features = features.view(batch_size, seq_len, -1)  # [Batch, Sequence, Features]

        # Pass the sequence of features through the GRU
        gru_out, _ = self.gru(features)  # [Batch, Sequence, Hidden_Size]

        # Take the output of the last time step
        final_output = gru_out[:, -1, :]  # [Batch, Hidden_Size]

        # Apply dropout before the fully connected layer
        final_output = self.fc_dropout(final_output)

        # Pass through the fully connected layer
        output = self.fc(final_output)  # [Batch, Num_Classes]
        return output

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# CNN + GRU Model with Dropouts
class r50gru(nn.Module):
    def __init__(self, num_classes, dropout, name='r50gru'):
        super(r50gru, self).__init__()

        # model_name
        self.name= name

        # Pre-trained CNN (e.g., ResNet)
        self.cnn = models.resnet50(pretrained=True)
        self.cnn = nn.Sequential(*list(self.cnn.children())[:-1])  # Remove the final classification layer

        # Dropout after CNN features
        self.cnn_dropout = nn.Dropout(p=dropout)

        # GRU for temporal modeling
        self.gru = nn.GRU(input_size=2048, hidden_size=256, num_layers=2, batch_first=True, dropout=dropout)

        # Fully connected layer for classification
        self.fc = nn.Linear(256, num_classes)

        # Dropout before final FC layer
        self.fc_dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        """
        Forward pass for the model.

        Args:
        - x: Input tensor with shape [Batch, Sequence, Channels, Height, Width]

        Returns:
        - output: Model predictions with shape [Batch, Num_Classes]
        """
        batch_size, seq_len, c, h, w = x.size()  # x: [Batch, Sequence, Channels, Height, Width]

        # Reshape input for CNN processing
        x = x.view(batch_size * seq_len, c, h, w)  # Combine Batch and Sequence dimensions: [Batch * Sequence, Channels, Height, Width]

        # Extract features using CNN
        features = self.cnn(x)  # [Batch * Sequence, 512, 1, 1] (ResNet-18 output)
        features = features.view(features.size(0), -1)  # Flatten: [Batch * Sequence, 512]

        # Apply CNN dropout
        features = self.cnn_dropout(features)

        # Reshape features back to sequence format for GRU
        features = features.view(batch_size, seq_len, -1)  # [Batch, Sequence, Features]

        # Pass the sequence of features through the GRU
        gru_out, _ = self.gru(features)  # [Batch, Sequence, Hidden_Size]

        # Take the output of the last time step
        final_output = gru_out[:, -1, :]  # [Batch, Hidden_Size]

        # Apply dropout before the fully connected layer
        final_output = self.fc_dropout(final_output)

        # Pass through the fully connected layer
        output = self.fc(final_output)  # [Batch, Num_Classes]
        return output

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# CNN + GRU Model with Dropouts
class r50lstm(nn.Module):
    def __init__(self,num_classes, dropout,name='r50lstm'):
        super(r50lstm, self).__init__()

        # model_name
        self.name= name

        # Pre-trained CNN (e.g., ResNet)
        self.cnn = models.resnet50(pretrained=True)
        self.cnn = nn.Sequential(*list(self.cnn.children())[:-1])  # Remove the final classification layer

        # Dropout after CNN features
        self.cnn_dropout = nn.Dropout(p=dropout)

        # GRU for temporal modeling
        self.gru = nn.LSTM(input_size=2048, hidden_size=256, num_layers=2, batch_first=True, dropout=dropout)

        # Fully connected layer for classification
        self.fc = nn.Linear(256, num_classes)

        # Dropout before final FC layer
        self.fc_dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        """
        Forward pass for the model.

        Args:
        - x: Input tensor with shape [Batch, Sequence, Channels, Height, Width]

        Returns:
        - output: Model predictions with shape [Batch, Num_Classes]
        """
        batch_size, seq_len, c, h, w = x.size()  # x: [Batch, Sequence, Channels, Height, Width]

        # Reshape input for CNN processing
        x = x.view(batch_size * seq_len, c, h, w)  # Combine Batch and Sequence dimensions: [Batch * Sequence, Channels, Height, Width]

        # Extract features using CNN
        features = self.cnn(x)  # [Batch * Sequence, 512, 1, 1] (ResNet-18 output)
        features = features.view(features.size(0), -1)  # Flatten: [Batch * Sequence, 512]

        # Apply CNN dropout
        features = self.cnn_dropout(features)

        # Reshape features back to sequence format for GRU
        features = features.view(batch_size, seq_len, -1)  # [Batch, Sequence, Features]

        # Pass the sequence of features through the GRU
        gru_out, _ = self.gru(features)  # [Batch, Sequence, Hidden_Size]

        # Take the output of the last time step
        final_output = gru_out[:, -1, :]  # [Batch, Hidden_Size]

        # Apply dropout before the fully connected layer
        final_output = self.fc_dropout(final_output)

        # Pass through the fully connected layer
        output = self.fc(final_output)  # [Batch, Num_Classes]
        return output

In [24]:
# shutil.rmtree('/kaggle/working')

In [25]:
# temp = read_pickle(train_path_lists[0])

In [26]:
# vid = temp['video']

In [27]:
# vid_1 = vid[:4]

In [28]:
# vid_1.to(torch.float16).to(device)

In [21]:
num_classes = 262

In [24]:
# model = r18gru(num_classes=num_classes,dropout=0.1).to(device)
model = r3d(num_classes).to(device)
# model = r50lstm(num_classes).to(device, dropout=0.1)
# model = r50gru(num_classes).to(device)
# model = r18lstm(num_classes).to(device)
# model = mv3gru(num_classes).to(device)
# model = nn.DataParallel(model)

AssertionError: Torch not compiled with CUDA enabled

In [41]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

# Define the scheduler without the verbose parameter
scheduler = ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.1,
    patience=5,
    min_lr=1e-6  # Set a minimum learning rate
)


# Define the loss function
loss_fn = nn.CrossEntropyLoss()

In [ ]:
trainer = Trainer(device=device,optimizer=optimizer,loss_fn=loss_fn,save=True, scheduler=scheduler, model=model,train_path_list=train_path_lists,test_path_list=test_path_lists)
epochs = 100
for epoch in range(epochs):
    print("Train:")
    trainer.train_step_pickle(epoch=epoch)
    print("Test all:")
    trainer.test_step_pickle(epoch=epoch)

Train:


<ipython-input-6-2324195e5bc1>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(path)


In [ ]:
# Plotting Confusion Matrix
'''
from sklearn.metrics import ConfusionMatrixDisplay
import numpy as np
from sklearn.metrics import confusion_matrix

y_true = []
y_preds = []

model.eval()
with torch.no_grad():
  for X, y in test_preloaded:
    X, y = X.to(device).float(), y.to(device)


    y_pred = model(X)
    y_pred_class = y_pred.argmax(dim=1)

    y_true.extend(y.cpu().numpy())
    y_preds.extend(y_pred_class.cpu().numpy())
plt.figure(figsize=(1000, 1000))
cm = confusion_matrix(y_true, y_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(50))
disp.plot(cmap="viridis")  # Customize colormap if needed
plt.xticks(fontsize=6, rotation=45, ha="right")
plt.yticks(fontsize=6)
plt.show()
import seaborn as sns
def cm_disp(cm):
    mask = np.zeros_like(cm)
    mask[np.triu_indices_from(mask)] = True
    sns.heatmap(cm, mask=mask, vmax=.3, square=True,  cmap="YlGnBu")
    plt.show()
cm_disp(cm)
'''

In [ ]:
from IPython.display import FileLink
FileLink('resnet_pretrained_acc=88.74_loss=0.2813_epoch=30_real.pth')  # can be used to get the download link for the model, especially useful when using kaggle

In [ ]:
# Load checkpoint for model1 (r18gru)
# model1 = r18gru(num_classes=50, dropout=0.1).to(device)
# model1 = nn.DataParallel(model1)  # Distribute model across GPUs

# checkpoint1 = torch.load('/kaggle/input/models/resnet_pretrained_GRU_acc88.74_loss0.2813_epoch30_real.pth')
# model1.load_state_dict(checkpoint1['model_state_dict'])

# Load checkpoint for model2 (r3d_18)
model2 = r3d(50).to(device)
# model2 = nn.DataParallel(model2)

checkpoint2 = torch.load('/kaggle/input/models/r3d_18_pretrained_acc89.18_loss0.7433_epoch15_real.pth')
model2.load_state_dict(checkpoint2['model_state_dict'])


In [ ]:
y_pred = []
y_true = []
model1.eval()
model2.eval()
with torch.no_grad():
    for path in test_path_all:
        tensor = read_pickle(path)
        X, y = tensor['model'], tensor['label']
        y_pred_batch = 0
        # Convert the labels and predictions to numpy arrays
        y_true_batch = y.tolist()
        #y_pred_batch += model1(X.to(torch.float32))
        y_pred_batch += model2(X.to(torch.float32))
        #y_pred_batch /= 2
        # Concatenate the current batch predictions and labels
        y_true.extend(y_true_batch)  # Wrap arrays in a tuple
        y_pred.extend(y_pred_batch.argmax(dim=1).tolist())

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
types= ['micro', 'macro', 'weighted']
for ty in types:
    print(ty)
    # Assuming y_true and y_pred are your ground truth and predictions
    precision = precision_score(y_true, y_pred, average=ty)  # Use 'binary' for binary classification
    recall = recall_score(y_true, y_pred, average=ty)        # Use 'macro' or 'micro' for multiclass if needed
    f1 = f1_score(y_true, y_pred, average=ty) # 'weighted' handles class imbalance
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print("***************")
print(f"Acc = {accuracy_score(y_true, y_pred):.4f}")


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
cm = confusion_matrix(y_true, y_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(50))
disp.plot(cmap="viridis")  # Customize colormap if needed
plt.xticks(fontsize=6, rotation=45, ha="right")
plt.yticks(fontsize=6)
plt.show()